In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
from torchvision import datasets, transforms
from kymatio.torch import Scattering2D
import torchvision.transforms.functional as functional
import numpy as np
from kymatio.torch import Scattering2D
from PIL import Image
import os, re, sys
import umap
import ImageFolderWithPaths as IF
import s2Net_orientation as network
import data_setup as DS
import network_function as NF
import matplotlib.pyplot as plt
import numpy as np

In [6]:
class_names = ['IM', ]  # class of image ('apple, orange, etc')
s2 = {'name': [], 'par': {}, 'per': {}, 'quart': {}}

n_cores = 0
im_size = (256, 256)
J = 5  # number of scales
L = 8  # number of orientations

j1 = 1
layer1_orientation = 1


data_dir = "Visualize_EG"

In [7]:
class s2Net(nn.Module):
    def __init__(self, J, L, layer1_orientation, j1):
        super(s2Net, self).__init__()

        self.l1 = layer1_orientation

        self.norm_index = []

        # Define index for parallel and perpendicular filters

        self.per_layer1_index = []
        self.per_layer2_index = []
        self.par_layer1_index = []
        self.par_layer2_index = []
        self.quarter_layer1_index = []
        self.quarter_layer2_index = []

        for j2 in range(j1 + 1, J):
            for l2 in range(L):

                coeff_index_layer2 = layer1_orientation * L * (J - j1 - 1) + l2 + L * (j2 - j1 - 1) + (L ** 2) * \
                                     (j1 * (J - 1) - j1 * (j1 - 1) // 2)

                coeff_index_layer1 = layer1_orientation + j1 * L

                # parallel
                if layer1_orientation == l2:
                    self.par_layer1_index.append(coeff_index_layer1)
                    self.par_layer2_index.append(coeff_index_layer2)
                # perpendicular
                if l2 == (layer1_orientation + (L / 2)) or l2 == (layer1_orientation - (L / 2)):
                    self.per_layer1_index.append(coeff_index_layer1)
                    self.per_layer2_index.append(coeff_index_layer2)
                # 45 degrees
                if l2 == (layer1_orientation + (L // 4)) or l2 == (layer1_orientation - (L // 4)):
                    self.quarter_layer1_index.append(coeff_index_layer1)
                    self.quarter_layer2_index.append(coeff_index_layer2)

    def forward(self, x):
        x = x.squeeze()
        sz = x.shape

        # Average pooling across different spatial location
        sti = x.mean((len(sz) - 2, len(sz) - 1))  # 1d array of values

        scat_coeffs_parallel_order_2 = sti[self.par_layer2_index,] / sti[self.par_layer1_index,]
        scat_coeffs_per_order_2 = sti[self.per_layer2_index,] / sti[self.per_layer1_index,]
        scat_coeffs_quart_order_2 = sti[self.quarter_layer2_index,] / sti[self.quarter_layer1_index,]

        # Return normalized s2 coefficient - activation
        return  scat_coeffs_parallel_order_2

In [8]:
dataloaders, device, dataset_sizes, scattering = DS.setup(im_size, class_names, data_dir, n_cores, J, L)

In [9]:
if torch.cuda.is_available():
        scattering = scattering.cuda()

In [11]:
s2_Net = s2Net(J, L, layer1_orientation, j1)

In [12]:
with torch.no_grad():
    for label in class_names:
        print(label)
        inputs = dataloaders[label]
        for i, ele in enumerate(inputs, 0):
            data, n1, path = ele
            # Get the name of the image from the path
            label = path[0]
            label = re.split("/", label)
            label = label[-1]

            # Load image into device
            data = data.to(device)

            # Run network
            output_parallel, output_per, output_quart = s2_Net(scattering(data))

IM


In [13]:
print(output_parallel)

tensor(0.0993)


In [17]:
output_parallel.shape()

TypeError: 'torch.Size' object is not callable